In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
#ws.get_details()

Ready to use Azure ML 1.42.0 to work with tcp0044aedcmluc001dtmlws


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "tcp0044uc001cc01"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


In [3]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


In [4]:
%%writefile $experiment_folder/ams_score.py
from __future__ import print_function
import pandas as pd
import numpy as np
import sklearn
from sklearn.covariance import EllipticEnvelope
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import os
from datetime import date
import re
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARMA
from joblib import Parallel, delayed
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.api import VAR
import joblib
import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import pandas as pd
import numpy as np
from sklearn.covariance import EllipticEnvelope
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import plotly.graph_objects as go
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import train_test_split
import shap 
sns.set_style('darkgrid')
from sklearn.decomposition import PCA
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
from matplotlib import pyplot
import plotly.graph_objs as go
from sklearn.svm import OneClassSVM
from numpy import where
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from azureml.core import Workspace, Dataset
from azureml.core.run import Run
from azureml.core.run import Run
from azureml.core.authentication import MsiAuthentication 

workspace = Run.get_context().experiment.workspace
run = Run.get_context()
ws = run.experiment.workspace



def init():
  
    global RandomforestBusiness_Machine_to_Machine_model,RandomforestBusinessPostpaid_Handheld_model,RandomforestBusinessPostpaidMobileBroadbandmodel,RandomforestBusinessPrepaid_Handheld_model,RandomforestBusinessPrepaidMobileBroadbandmodel,RandomforestConsumerMachinetoMachinemodel,RandomforestConsumerPostpaidHandheldmodel,RandomforestConsumerPostpaidMobileBroadbandmodel,RandomforestConsumerPrepaidHandheldmodel,RandomforestConsumerPrepaidMobileBroadbandmodel,RandomforestTelstraEnterpriseMachinetoMachinemodel,RandomforestTelstraEnterprisePostpaidHandheldmodel,RandomforestTelstraEnterprisePostpaidMobileBroadbandmodel,RandomforestTelstraEnterprisePrepaidHandheldmodel,RandomforestTelstraEnterprisePrepaidMobileBroadbandmodel

    category=["Business-Machine to Machine","Business-Postpaid Handheld","Business-Postpaid Mobile Broadband","Business-Prepaid Handheld","Business-Prepaid Mobile Broadband","Consumer-Machine to Machine","Consumer-Postpaid Handheld","Consumer-Postpaid Mobile Broadband","Consumer-Prepaid Handheld","Consumer-Prepaid Mobile Broadband","Telstra Enterprise-Machine to Machine","Telstra Enterprise-Postpaid Handheld","Telstra Enterprise-Postpaid Mobile Broadband","Telstra Enterprise-Prepaid Handheld","Telstra Enterprise-Prepaid Mobile Broadband"]
    for i in category:
        
        k=i.replace(" ", "_")
        k=k.replace("-","_")
        modelname = 'Randomforest'+k+'_model'
        modelnam_type = 'Randomforest'+k+'_model.pkl'
        saved_model = 'Randomforest'+k+'_model'
        #model_path = '../Randomforest'+k+'_model.pkl'
        model_path = Model.get_model_path(modelname,version=None)
        # Deserialize the model files back into scikit-learn models.
        saved_model = joblib.load(model_path)

    print("models retrieved")
    



def run(mini_batch):

    SIO_data_salescube = Dataset.get_by_name(ws, name='SIO_salescube')
    SIO_data_salescube = SIO_data_salescube.to_pandas_dataframe()

    Activity_data_salescube = Dataset.get_by_name(ws, name='activity_salescube')
    Activity_data_salescube = Activity_data_salescube.to_pandas_dataframe()

    SIO_data_davinci = Dataset.get_by_name(ws, name='SIO_davinci')
    SIO_data_davinci = SIO_data_davinci.to_pandas_dataframe()

    Activity_data_davinci = Dataset.get_by_name(ws, name='activity_davinci')
    Activity_data_davinci = Activity_data_davinci.to_pandas_dataframe()

    SIO_data_salescube = SIO_data_salescube.iloc[2:].reset_index()
    SIO_data_salescube.columns = SIO_data_salescube.iloc[0]

    SIO_data_salescube = SIO_data_salescube.iloc[1:].reset_index()

    SIO_data_salescube.drop(['index',2], inplace=True, axis=1)

    SIO_data_salescube['Closing SIOs'] = SIO_data_salescube['Closing SIOs'].replace(',', '', regex=True).astype(float)
    SIO_data_salescube['Opening SIOs'] = SIO_data_salescube['Opening SIOs'].replace(',', '', regex=True).astype(float)
    SIO_data_salescube['SIO_Growth']=SIO_data_salescube['Closing SIOs']-SIO_data_salescube['Opening SIOs']
    SIO_data_salescube.drop(['Srvc Type Group','Opening SIOs'], inplace=True, axis=1)
    SIO_data_salescube = SIO_data_salescube.rename(columns = {'Week Ending':'Week_Ending','Business Unit': 'Segment','Closing SIOs':'Closing SIO'}, inplace = False)

    SIO_data_salescube['Srvc Type Category'] = SIO_data_salescube['Srvc Type Category'].fillna('No Value')
    SIO_data_salescube['Srvc Type Sub Category'] = SIO_data_salescube['Srvc Type Sub Category'].fillna('No Value')
    SIO_data_salescube['Service Type'] = SIO_data_salescube['Service Type'].fillna('No Value')
    SIO_data_salescube['SIO_Growth'] = SIO_data_salescube['SIO_Growth'].fillna(0)
    SIO_data_salescube['Closing SIO'] = SIO_data_salescube['Closing SIO'].fillna(0)

    for i in range(len(SIO_data_salescube)):
        if (SIO_data_salescube.loc[i,'Srvc Type Sub Category']=="WBB Cards - Mobile Pre"):
            SIO_data_salescube.loc[i,'Product'] = "Prepaid Mobile Broadband"
        elif (SIO_data_salescube.loc[i,'Srvc Type Sub Category']=="Postpaid Telstra Mobile Broadband"):
            SIO_data_salescube.loc[i,'Product'] = "Postpaid Mobile Broadband"
        elif (SIO_data_salescube.loc[i,'Srvc Type Sub Category']=="Mobiles - Handheld Prepaid"):
            SIO_data_salescube.loc[i,'Product'] = "Prepaid Handheld"
        elif (SIO_data_salescube.loc[i,'Srvc Type Sub Category']=="Mobiles - Handheld Postpaid"):
            SIO_data_salescube.loc[i,'Product'] = "PostpaidHandheld"
        elif (SIO_data_salescube.loc[i,'Srvc Type Category']=="Mobiles - Telemetry"):
            SIO_data_salescube.loc[i,'Product'] = "Machine to Machine"
        else:
            SIO_data_salescube.loc[i,'Product'] = "Others"

    SIO_data_salescube.drop(['Srvc Type Category','Srvc Type Sub Category','Service Type'], inplace=True, axis=1)

    SIO_data_salescube['Segment'] = SIO_data_salescube['Segment'].replace({'TELSTRA BUSINESS GROUP': 'Business', 'TELSTRA CONSUMER': 'Consumer', 'Segment Sales': 'Telstra Enterprise'})

    SIO_data_davinci = SIO_data_davinci.iloc[:-1]

    SIO_data_davinci.drop(['Product Level 2','Opening SIO'], inplace=True, axis=1)
    SIO_data_davinci = SIO_data_davinci.rename(columns = {'SIO Growth':'SIO_Growth','Row Labels':'Week_Ending','Product Level 3':'Product_Level3','Product Level 4':'Product_Level4','Business Unit Name': 'Segment'}, inplace = False)

    for i in range(len(SIO_data_davinci)):
        if (SIO_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (SIO_data_davinci.loc[i,'Product_Level4']=="MBB Plan"):
            SIO_data_davinci.loc[i,'Product'] = "Prepaid Mobile Broadband"
        elif (SIO_data_davinci.loc[i,'Product_Level3']=="Postpaid Mobile") and (SIO_data_davinci.loc[i,'Product_Level4']=="MBB Plan"):
            SIO_data_davinci.loc[i,'Product'] = "Postpaid Mobile Broadband"
        elif (SIO_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (SIO_data_davinci.loc[i,'Product_Level4']=="Mobile Plan"):
            SIO_data_davinci.loc[i,'Product'] = "Prepaid Handheld"
        elif (SIO_data_davinci.loc[i,'Product_Level3']=="Postpaid Mobile") and (SIO_data_davinci.loc[i,'Product_Level4']=="Mobile Plan"):
            SIO_data_davinci.loc[i,'Product'] = "Postpaid Handheld"
        elif (SIO_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (SIO_data_davinci.loc[i,'Product_Level4']=="Boost Plan"):
            SIO_data_davinci.loc[i,'Product'] = "Prepaid Handheld"
        else:
            SIO_data_davinci.loc[i,'Product'] = "Others"

    SIO_data_davinci.drop(['Product_Level3','Product_Level4'], inplace=True, axis=1)

    SIO_data_davinci.Product.unique()

    df_SIO_both=pd.concat([SIO_data_salescube, SIO_data_davinci]).groupby(['Week_Ending','Product','Segment'], as_index=False)['SIO_Growth','Closing SIO'].sum()

    df_SIO_both['Week_Ending'] = df_SIO_both['Week_Ending'].str.replace("WE ", "")
    df_SIO_both['Week_Ending'] = pd.to_datetime(df_SIO_both['Week_Ending'], dayfirst = True)

    Activity_data_salescube = Activity_data_salescube.iloc[2:].reset_index()
    Activity_data_salescube.columns = Activity_data_salescube.iloc[0]

    Activity_data_salescube = Activity_data_salescube.iloc[1:].reset_index()

    Activity_data_salescube.drop(['index',2], inplace=True, axis=1)

    Activity_data_salescube['Event'] = Activity_data_salescube[['Event Category','Event Sub Category']].apply(lambda row: '-'.join(row.dropna()), axis=1)

    Activity_data_salescube = Activity_data_salescube.rename(columns = {'Week Ending':'Week_Ending','Business Unit': 'Segment'}, inplace = False)

    Activity_data_salescube['Event Category'] = Activity_data_salescube['Event Category'].fillna('No Value')
    Activity_data_salescube['Event Sub Category'] = Activity_data_salescube['Event Sub Category'].fillna('No Value')
    Activity_data_salescube['Event Sub Category Type'] = Activity_data_salescube['Event Sub Category Type'].fillna('No Value')
    Activity_data_salescube['Event Class'] = Activity_data_salescube['Event Class'].fillna('No Value')
    Activity_data_salescube['Total'] = Activity_data_salescube['Total'].fillna(0)
    Activity_data_salescube['Event'] = Activity_data_salescube['Event'].fillna('No Event')

    for i in range(len(Activity_data_salescube)):
        if (Activity_data_salescube.loc[i,'Srvc Type Sub Category']=="WBB Cards - Mobile Pre"):
            Activity_data_salescube.loc[i,'Product'] = "Prepaid Mobile Broadband"
        elif (Activity_data_salescube.loc[i,'Srvc Type Sub Category']=="Postpaid Telstra Mobile Broadband"):
            Activity_data_salescube.loc[i,'Product'] = "Postpaid Mobile Broadband"
        elif (Activity_data_salescube.loc[i,'Srvc Type Sub Category']=="Mobiles - Handheld Prepaid"):
            Activity_data_salescube.loc[i,'Product'] = "Prepaid Handheld"
        elif (Activity_data_salescube.loc[i,'Srvc Type Sub Category']=="Mobiles - Handheld Postpaid"):
            Activity_data_salescube.loc[i,'Product'] = "Postpaid Handheld"
        elif (Activity_data_salescube.loc[i,'Srvc Type Category']=="Mobiles - Telemetry"):
            Activity_data_salescube.loc[i,'Product'] = "Machine to Machine"
        else:
            Activity_data_salescube.loc[i,'Product'] = "Others"

    Activity_data_salescube.drop(['Srvc Type Category','Srvc Type Sub Category','Service Type'], inplace=True, axis=1)

    Activity_data_salescube['Segment'] = Activity_data_salescube['Segment'].replace({'TELSTRA BUSINESS GROUP': 'Business', 'TELSTRA CONSUMER': 'Consumer', 'Segment Sales': 'Telstra Enterprise'})

    Activity_data_salescube.drop(['Srvc Type Group'], inplace=True, axis=1)

    Activity_data_davinci['Event'] = Activity_data_davinci[['Activity Type Category','Activity Type Sub Category']].apply(lambda row: '-'.join(row.dropna()), axis=1)
    #Activity_data_davinci['Event']=Activity_data_davinci['Activity Type Category']+"-"+Activity_data_davinci['Activity Type Sub Category']+"-"+Activity_data_davinci['Activity Type Sub Category Type']+"-"+Activity_data_davinci['Activity Type Class']
    Activity_data_davinci.drop(['Product Level 2'], inplace=True, axis=1)
    Activity_data_davinci = Activity_data_davinci.rename(columns = {'Row Labels':'Week_Ending','Business Unit Name': 'Segment','Product Level 3':'Product_Level3','Product Level 4': 'Product_Level4','Activity Type Category':'Event Category','Activity Type Sub Category':'Event Sub Category','Activity Type Sub Category Type':'Event Sub Category Type','Activity Type Class':'Event Class'}, inplace = False)

    Activity_data_davinci = Activity_data_davinci.iloc[:-1]
    Activity_data_davinci = Activity_data_davinci.rename(columns = {'Units':'Total'}, inplace = False)

    for i in range(len(Activity_data_davinci)):
        if (Activity_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (Activity_data_davinci.loc[i,'Product_Level4']=="MBB Plan"):
            Activity_data_davinci.loc[i,'Product'] = "Prepaid Mobile Broadband"
        elif (Activity_data_davinci.loc[i,'Product_Level3']=="Postpaid Mobile") and (Activity_data_davinci.loc[i,'Product_Level4']=="MBB Plan"):
            Activity_data_davinci.loc[i,'Product'] = "Postpaid Mobile Broadband"
        elif (Activity_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (Activity_data_davinci.loc[i,'Product_Level4']=="Mobile Plan"):
            Activity_data_davinci.loc[i,'Product'] = "Prepaid Handheld"
        elif (Activity_data_davinci.loc[i,'Product_Level3']=="Postpaid Mobile") and (Activity_data_davinci.loc[i,'Product_Level4']=="Mobile Plan"):
            Activity_data_davinci.loc[i,'Product'] = "Postpaid Handheld"
        elif (Activity_data_davinci.loc[i,'Product_Level3']=="Prepaid Mobile") and (Activity_data_davinci.loc[i,'Product_Level4']=="Boost Plan"):
            Activity_data_davinci.loc[i,'Product'] = "Prepaid Handheld"
        else:
            Activity_data_davinci.loc[i,'Product'] = "Others"

    Activity_data_davinci.drop(['Product_Level3','Product_Level4'], inplace=True, axis=1)

    df_Activity_both=pd.concat([Activity_data_salescube, Activity_data_davinci]).groupby(['Week_Ending','Product','Segment','Event'], as_index=False)['Total'].sum()

    df_Activity_both['Week_Ending'] = df_Activity_both['Week_Ending'].str.replace("WE ", "")
    df_Activity_both['Week_Ending'] = pd.to_datetime(df_Activity_both['Week_Ending'], dayfirst = True)

    df_Activity_both = df_Activity_both[(df_Activity_both.Product != "Others") & (df_Activity_both.Segment != "Not Available") & (df_Activity_both.Segment != "Unknown") & (df_Activity_both.Event != "No Event")]

    df_Activity_both['Total'] = pd.to_numeric(df_Activity_both['Total'],errors='coerce')

    df_Activity_both_ML = df_Activity_both.pivot_table(index=['Week_Ending','Segment','Product'],columns='Event',values='Total').fillna(0)
    df_Activity_both_ML = df_Activity_both_ML.reset_index()
    df_activity_Segment_Product = df_Activity_both_ML.copy()
    df_activity_Segment_Product['Segment-Product'] = df_Activity_both_ML['Segment']+ '-' +df_Activity_both_ML['Product']
    df_activity_Segment_Product=df_activity_Segment_Product.drop(['Segment','Product'], axis=1)
    df_activity_Segment_Product=df_activity_Segment_Product.groupby(['Week_Ending','Segment-Product']).agg('sum')
    df_activity_Segment_Product=df_activity_Segment_Product.reset_index()

    df_SIO_both = df_SIO_both[(df_SIO_both.Product != "Others") & (df_SIO_both.Segment != "Not Available") & (df_SIO_both.Segment != "Unknown")]

    df_SIO_Segment_Product = df_SIO_both.copy()
    df_SIO_Segment_Product['Segment-Product'] = df_SIO_both['Segment']+ '-' +df_SIO_both['Product']
    df_SIO_Segment_Product=df_SIO_Segment_Product.drop(['Segment','Product'], axis=1)

    df_SIO_Segment_Product['SIO_Growth'] = pd.to_numeric(df_SIO_Segment_Product['SIO_Growth'],errors='coerce').fillna(0)
    df_SIO_Segment_Product=df_SIO_Segment_Product.groupby(['Week_Ending','Segment-Product']).agg({'SIO_Growth':'sum'})
    df_SIO_Segment_Product=df_SIO_Segment_Product.reset_index()

    # merge the activities and SIOs
    df_final=pd.merge(df_activity_Segment_Product, df_SIO_Segment_Product, on=['Week_Ending', 'Segment-Product'])

    def drop_allzero_columns(df):
        for i in df.columns:
            if ((df[i] == 0.0).all() == True ):
                df.drop([i], inplace=True, axis=1)
        return df

    df_final = drop_allzero_columns(df_final)
    df_final['SIO_Growth'] = pd.to_numeric(df_final['SIO_Growth'],errors='coerce').fillna(0)
    df3 = df_final.copy()

    # ### common functions for ARIMA

    def test_stationarity(ts_data, column='', signif=0.05, series=False):
        if series:
            adf_test = adfuller(ts_data, autolag='AIC')
        else:
            adf_test = adfuller(ts_data[column], autolag='AIC')
            p_value = adf_test[1]
            if p_value <= signif:
                test_result = "Stationary"
            else:
                test_result = "Non-Stationary"
            return test_result

    max_p, max_q = 5, 5

    def get_order_sets(n, n_per_set) -> list:
        n_sets = [i for i in range(n)]
        order_sets = [
            n_sets[i:i + n_per_set]
            for i in range(0, n, n_per_set)
        ]
        return order_sets

    def find_aic_for_model(data, p, q, model, model_name):
        try:
            msg = f"Fitting {model_name} with order p, q = {p, q}n"
            print(msg)
            if p == 0 and q == 1:
                # since p=0 and q=1 is already
                # calculated
                return None, (p, q)
            ts_results = model(data, order=(p, q)).fit(disp=False)
            curr_aic = ts_results.aic
            return curr_aic, (p, q)
        except Exception as e:
            f"""Exception occurred continuing {e}"""
            return None, (p, q)

    def find_best_order_for_model(data, model, model_name):
        p_ar, q_ma = max_p, max_q
        final_results = []
        ts_results = model(data, order=(0, 0, 1)).fit()
        min_aic = ts_results.aic
        final_results.append((min_aic, (0, 0, 1)))
        # example if q_ma is 6
        # order_sets would be [[0, 1, 2, 3, 4], [5]]
        order_sets = get_order_sets(q_ma, 5)
        for p in range(0, p_ar):
            for order_set in order_sets:
                # fit the model and find the aic
                results = Parallel(n_jobs=len(order_set), prefer='threads')(
                    delayed(find_aic_for_model)(data, p, q, model, model_name)
                    for q in order_set
                )
                final_results.extend(results)
        results_df = pd.DataFrame(
                final_results,
                columns=['aic', 'order']
        )
        min_df = results_df[
            results_df['aic'] == results_df['aic'].min()
        ]
        min_aic = min_df['aic'].iloc[0]
        min_order = min_df['order'].iloc[0]
        return min_aic, min_order, results_df

    def find_anomalies(squared_errors,Z):
        threshold = np.mean(squared_errors) + Z*np.std(squared_errors)
        predictions = (squared_errors >= threshold).astype(int)
        return predictions, threshold

    def scoring_function_arima(model, df):
        # 1. model predictions
        squared_errors = model.resid ** 2
        # 2. Use a regression metric, e.g. MAE loss
        anomaly_scores = squared_errors # i.e. anomaly score
        return anomaly_scores

    def differencing(data, column, order):
        differenced_data = data[column].diff(order)
        differenced_data.fillna(differenced_data.mean(), inplace=True)
        return differenced_data

    def iqr_bounds(scores):
        q1 = scores.quantile((factor/2))
        q3 = scores.quantile(1-(factor/2))
        #iqr = q3 - q1
        lower_bound=(q1)
        upper_bound=(q3)
        return lower_bound,upper_bound

    def classify_anomalies(df,metric_name):
        df['metric_name']=metric_name
        df = df.sort_values(by='Week_Ending', ascending=False)
        #Shift actuals by one timestamp to find the percentage chage between current and previous data point
        df['shift'] = df['actuals'].shift(-1)
        df['percentage_change'] = ((df['actuals'] - df['shift']) / df['actuals']) * 100
        return df

    df_arima=df3[['SIO_Growth','Week_Ending','Segment-Product']]
    
    df3 = df3.loc[df3["Week_Ending"] > '2021-09-24']
    df_arima = df_arima.loc[df_arima["Week_Ending"] > '2021-09-24']

    import statsmodels.api as sm
    arima_results = pd.DataFrame()
    category = df_arima['Segment-Product'].unique()
    for i in category:
        #print(i)
        df_arima_model=df_arima[df_arima['Segment-Product']== i]
        df_arima_model.set_index('Week_Ending', drop=True, inplace=True)
        ts_data = df_arima_model.drop(['Segment-Product'],axis=1)
        
        ts_data = ts_data.astype('float64')
        
        k=i.replace(" ", "_")
        k=k.replace("-","_")
        modelname = 'Arima'+k+'_model'
        model_path = Model.get_model_path(modelname,version=None)
        # Deserialize the model files back into scikit-learn models.
        saved_model = joblib.load(model_path)
        df5 = scoring_function_arima(saved_model, ts_data)
        predictions, threshold = find_anomalies(df5,0)
        df_arima_model['predictions'] = predictions
        data4 = df_arima_model.copy()
        data4['change percentage'] = data4['SIO_Growth'].pct_change()
        data4['change percentage'] = abs(data4['change percentage'])
        data4['change percentage'] = data4['change percentage'].replace(np.inf, max(data4['change percentage']))
        data4 = data4.fillna(0)
        avgprcntgechnge = data4['change percentage'].mean()
        n=50
        data5 = data4.iloc[-n:]
        count = (data5[data5['change percentage'] > avgprcntgechnge]['change percentage'].count())
        contaminationfactor = count/n 
        factor = max(0.1,contaminationfactor)
        lower_bound,upper_bound=iqr_bounds(df_arima_model['SIO_Growth'])
        df_arima_model['Anomaly_Label_IQR']=0
        df_arima_model['Anomaly_Label_IQR']=(df_arima_model['SIO_Growth'] < lower_bound) |(df_arima_model['SIO_Growth'] > upper_bound)
        df_arima_model['Anomaly_Label_IQR']=df_arima_model['Anomaly_Label_IQR'].astype(int)  
        
        df_arima_model['Anomaly_Label_IQR'] = df_arima_model['Anomaly_Label_IQR'].replace(np.inf, 0)
        df_arima_model['predictions'] = df_arima_model['predictions'].replace(np.inf, 0)
        df_arima_model['Anomaly_Label_IQR'] = df_arima_model['Anomaly_Label_IQR'].fillna(0)
        df_arima_model['predictions'] = df_arima_model['predictions'].fillna(0)

        #test_scores = emmv_scores(arma_fit,ts_data,scoring_function_arima)
        f1_IQR = f1_score(df_arima_model['Anomaly_Label_IQR'], df_arima_model['predictions'], average='binary')
        precision_IQR = precision_score(df_arima_model['Anomaly_Label_IQR'], df_arima_model['predictions'], average='binary')
        recall_IQR = recall_score(df_arima_model['Anomaly_Label_IQR'], df_arima_model['predictions'], average='binary')
        tn_IQR, fp_IQR, fn_IQR, tp_IQR = confusion_matrix(df_arima_model['Anomaly_Label_IQR'], df_arima_model['predictions'], labels=[0, 1]).ravel()

        df_arima_model['f1_IQR']=f1_IQR
        df_arima_model['precision_IQR'] = precision_IQR
        df_arima_model['recall_IQR'] = recall_IQR
        df_arima_model['tn_IQR'] = tn_IQR
        df_arima_model['fp_IQR'] = fp_IQR
        df_arima_model['fn_IQR'] = fn_IQR
        df_arima_model['tp_IQR'] = tp_IQR

        arima_results = arima_results.append(df_arima_model)

    
    # Create a folder
    batch_folder = './Output-inference'
    os.makedirs(batch_folder, exist_ok=True)
    print("Folder created!")

    local_path1 = './Output-inference/Arima_results.csv'    
    arima_results.to_csv(local_path1,index=False)

    # Using random forest and RNN and SHAP with anomalies given by Arima as label

    arima_results = arima_results.reset_index()

    arima_results['Week_Ending'] = pd.to_datetime(arima_results['Week_Ending'], dayfirst = True)

    # merge the activities and SIOs
    df_logreg = pd.merge(df3, arima_results, on=['Week_Ending', 'Segment-Product'])

    df_logreg.drop(['SIO_Growth_y','Anomaly_Label_IQR','f1_IQR','precision_IQR','recall_IQR','tn_IQR','fp_IQR','fn_IQR','tp_IQR'], inplace=True, axis=1)
    df_logreg = df_logreg.rename(columns = {'SIO_Growth_x':'SIO_Growth'}, inplace = False)


    #df_logreg = df3.loc[df3["Week_Ending"] > '2021-09-24']

    category = df_logreg['Segment-Product'].unique()

    # #### RUN random forest classifier 

    RandomForest_results = pd.DataFrame()
    Arima_featureimportance_global = pd.DataFrame()
    Arima_featureimportance_local = pd.DataFrame()
    Arima_Activity_percentage_change = pd.DataFrame()

    for i in category:
        
        data=df_logreg[df_logreg['Segment-Product']== i]
        
        data = data.drop(['Segment-Product'],axis=1)
        data = data.groupby(['Week_Ending']).sum()
        
        data4 = data.copy()
        data4['change percentage'] = data4['SIO_Growth'].pct_change()
        data4['change percentage'] = abs(data4['change percentage'])
        data4['change percentage'] = data4['change percentage'].replace(np.inf, max(data4['change percentage']))
        data4 = data4.fillna(0)
        avgprcntgechnge = data4['change percentage'].mean()
        n=50
        data6 = data4.iloc[-n:]

        count = (data6[data6['change percentage'] > avgprcntgechnge]['change percentage'].count())
        contaminationfactor = count/n

        factor = max(0.1,contaminationfactor)
        
        high_thresh=(1.0+factor)
        low_thresh=(1.0-factor)

        # split into X and y
        X = data4.drop(['predictions','change percentage'], axis=1)
        y = data4['predictions']
        X = X.reset_index()
        y = y.reset_index()
        X = X.groupby(['Week_Ending']).sum()
        y = y.groupby(['Week_Ending']).sum()
        
        data7=X.copy()
        data7['change percentage'] = data7['SIO_Growth'].pct_change()
        data7['change percentage'] = abs(data7['change percentage'])
        data7 = data7.fillna(0)
        avgprcntgechnge1 = data7['change percentage'].mean()
        n1=50
        data8 = data7.iloc[-n1:]
        count1 = (data8[data8['change percentage'] > avgprcntgechnge1]['change percentage'].count())
        contaminationfactor1 = count1/n1
        factor1 = max(0.1,contaminationfactor1)
        high_thresh1=(1.0+factor1)
        low_thresh1=(1.0-factor1)
        
        X = X.drop(['SIO_Growth'], axis=1)
        
        k=i.replace(" ", "_")
        k=k.replace("-","_")
        modelname = 'Randomforest'+k+'_model'
        #modelnam_type = 'Randomforest'+k+'_model.pkl'
        #model_path = '../Randomforest'+k+'_model.pkl'
        model_path = Model.get_model_path(modelname,version=None)
        # Deserialize the model files back into scikit-learn models.
        saved_model = joblib.load(model_path)
        data1=X.copy()
        data1['anomaly_RF'] = saved_model.predict(X)
        data1['anomaly_arima']=y['predictions']

        # get the local feature importances at each index level for every segment-product combination using SHAP
        data = data.drop(['SIO_Growth'], axis=1)
        explainer = shap.TreeExplainer(saved_model, data=data[data.columns])
        shapvs = explainer.shap_values(data[data.columns], check_additivity=True) # check should be default

        data3=pd.DataFrame()
        for l in range(0,len(data)):
            data3 = data3.append(dict(zip(data.columns, shapvs[l])), ignore_index=True) 
            
        data_2=data1.copy()
        data_2['SIO_Growth'] = data7['SIO_Growth']
        data_2=data_2.reset_index()
        features_perchange=pd.DataFrame()
        for m in range(1,len(data_2.columns)-1):
            test_df=pd.DataFrame()
            test_df['Week_Ending']=data_2['Week_Ending']
            test_df['actuals']=data_2.iloc[:,m:m+1]
            test_df=classify_anomalies(test_df,data_2.columns[m])
            features_perchange = features_perchange.append(test_df)
        
        for n in data1.index:
            if (data1.loc[n,'anomaly_RF'] > 0.1):
                data1.loc[n,'anomaly_RF'] = 1
            else:
                data1.loc[n,'anomaly_RF'] = 0
        
        precision_IQR = precision_score(data1['anomaly_arima'], data1['anomaly_RF'])
        recall_IQR = recall_score(data1['anomaly_arima'], data1['anomaly_RF'])
        f1_IQR = f1_score(data1['anomaly_arima'], data1['anomaly_RF'])
        confusion_IQR = confusion_matrix(data1['anomaly_arima'], data1['anomaly_RF'])
        tn_IQR, fp_IQR, fn_IQR, tp_IQR = confusion_matrix(data1['anomaly_arima'], data1['anomaly_RF'], labels=[0, 1]).ravel()

        data1['SIO_Growth'] = data7['SIO_Growth']    
        # data1['Week_Ending'] = data_2['Week_Ending']  
        #data1 = data1.set_index('Week_Ending')
       
        data1['Segment-Product'] = i
        data1['precision_IQR'] = precision_IQR
        data1['recall_IQR'] = recall_IQR
        data1['f1_IQR'] = f1_IQR
        data1['tn_IQR'] = tn_IQR
        data1['fp_IQR'] = fp_IQR
        data1['fn_IQR'] = fn_IQR
        data1['tp_IQR'] = tp_IQR
        data3['Segment-Product'] = i
        data3['Week_Ending'] = data1.index
        #data3 = data3.set_index('Week_Ending')
        data1=data1.reset_index()
        features_perchange['Segment-Product'] = i
        features_perchange = features_perchange.set_index('Week_Ending')
        RandomForest_results = RandomForest_results.append(data1)
        Arima_featureimportance_local = Arima_featureimportance_local.append(data3)
        Arima_Activity_percentage_change = Arima_Activity_percentage_change.append(features_perchange)

        local_path2 = './Output-inference/RandomForest_results.csv'
        RandomForest_results.to_csv(local_path2,index=False)

        local_path3 = './Output-inference/Arima_featureimportance_local.csv'
        Arima_featureimportance_local.to_csv(local_path3,index=False)

        local_path4 = './Output-inference/Arima_Activity_percentage_change.csv'
        Arima_Activity_percentage_change.to_csv(local_path4,index=False)

        # Upload the files to the default datastore
        print("Uploading files to datastore...")
        default_ds = ws.get_default_datastore()
        default_ds.upload(src_dir="Output-inference", target_path="Output-inference", overwrite=True, show_progress=True)

        # Register a datasets for output data
        batch_data_set1 = Dataset.Tabular.from_delimited_files(path=(default_ds, 'Output-inference/Arima_featureimportance_local.csv'), validate=False)
        batch_data_set1 = batch_data_set1.register(workspace=ws, 
                                                    name='Arima_featureimportance_local',
                                                    description='Arima_featureimportance_local',
                                                    create_new_version=True)
        batch_data_set2 =Dataset.Tabular.from_delimited_files(path=(default_ds, 'Output-inference/Arima_results.csv'), validate=False)
        batch_data_set2 = batch_data_set2.register(workspace=ws, 
                                                    name='Arima_results',
                                                    description='Arima_results',
                                                    create_new_version=True)


    return arima_results

                    





Overwriting batch_pipeline/ams_score.py


In [5]:
%%writefile $experiment_folder/team_script.py
from azureml.core import Workspace, Dataset
import pandas as pd
from azureml.core import Workspace, Dataset
from azureml.core.run import Run
from azureml.core.authentication import MsiAuthentication 
workspace = Run.get_context().experiment.workspace
run = Run.get_context()
ws = run.experiment.workspace




Arima_results = Dataset.get_by_name(workspace, name='Arima_results')
Arima_results = Arima_results.to_pandas_dataframe()
#Arima_results

arima_recents = Arima_results.loc[Arima_results.Week_Ending == Arima_results['Week_Ending'].max()]
#RandomForest_results

arima_recents = arima_recents.loc[arima_recents.predictions == 1]
#arima_recents

if len(arima_recents)!=0:
    res  = "Anomaly Detected"
    colour = "warning"
else:
    res = "No Anomaly Detected"
    colour = "good"

arima_recents = arima_recents.reset_index()
var1 = arima_recents['Week_Ending'][0]
res1 = str(var1.date())
#res1

arima_recents = arima_recents.loc[arima_recents.predictions == 1]
arima_recents = arima_recents.reset_index(drop=True)
Anamalous_segments_list = arima_recents['Segment-Product'].tolist()
table1 = pd.DataFrame()
#table1['Week_Ending'] = arima_recents['Week_Ending']
table1['Category'] = arima_recents['Segment-Product']
Anamalous_segments = " & ".join(map(str,Anamalous_segments_list))
res2 = "in "+f'{Anamalous_segments}'
#table1
#res2

import requests
import json
url = "https://teamtelstra.webhook.office.com/webhookb2/af091050-7f47-460f-b960-fbf683c74f16@49dfc6a3-5fb7-49f4-adea-c54e725bb854/IncomingWebhook/0e1b6ac2c6a845a1b7248afbbb9d105f/7282c1e3-77c4-42bf-802b-6d41373db2cc"
payload = {
    "type":"message",
    "attachments":[
        { 
            "contentType":"application/vnd.microsoft.card.adaptive",
            "content":{
                "type": "AdaptiveCard",
                "$schema": "http://adaptivecards.io/schemas/adaptive-card.json",
                "version": "1.0",
                "msTeams": { "width": "full" },
                "body": [
                    {
                        "type": "TextBlock",
                        "text": "**AMS - SIO Growth weekly anamolies**",
                        "wrap": True,
                        "size": "Large",
                        "spacing": "None",
                        "color": "Accent",
                        "weight": "bolder"
                    },
                    {
                        "type": "TextBlock",
                        "text": "Anomaly Detection for week ending "+res1+" at Business Segment/Product level",
                        "size": "small",
                        "color": "default",
                        "spacing" : "none",
                        "wrap" : True
                    },
                    {
                        "type": "Container",
                        "items": [
                            {
                                "type": "TextBlock",
                                "text": res+" ⚠️ ",
                                "size": "small",
                                "spacing": "default",
                                "color" : colour,
                                "weight" : "Bolder",
                                "wrap" : True
                            },
                            {
                                "type": "TextBlock",
                                "text": res2,
                                "size": "small",
                                "spacing": "default",
                                "color" : "good",
                                "weight" : "Bolder",
                                "wrap" : True   
                            }
                        ]
                    }
                ],
                "actions": [
                    {
                        "type": "Action.OpenUrl",
                        "title": "Detailed PowerBi Report",
                        "url":"https://app.powerbi.com/groups/5b205159-a3d0-4e0a-ab00-108e68d0b631/reports/79d9c943-8c8d-4f10-a5ee-9dcb58eb77d9/ReportSectionfad851ac51100a0d1502"
                    }
                ]
            }
        }
    ]
}  


headers = {
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.text.encode('utf8'))




Overwriting batch_pipeline/team_script.py


In [6]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)


IsolationforestConsumer_Moves_model version: 1
1classvmTelstra_Enterprise_Postpaid_Handheld_model version: 2
1classvmConsumer_Postpaid_Handheld_model version: 2
1classvmBusiness_Postpaid_Handheld_model version: 2
1classvmTelstra_Enterprise_Machine_to_Machine_model version: 2
1classvmTelstra_Enterprise_Prepaid_Mobile_Broadband_model version: 2
1classvmTelstra_Enterprise_Prepaid_Handheld_model version: 2
1classvmTelstra_Enterprise_Postpaid_Mobile_Broadband_model version: 2
1classvmConsumer_Prepaid_Mobile_Broadband_model version: 2
1classvmConsumer_Prepaid_Handheld_model version: 2
1classvmConsumer_Postpaid_Mobile_Broadband_model version: 2
1classvmConsumer_Machine_to_Machine_model version: 2
1classvmBusiness_Prepaid_Mobile_Broadband_model version: 2
1classvmBusiness_Prepaid_Handheld_model version: 2
1classvmBusiness_Postpaid_Mobile_Broadband_model version: 2
1classvmBusiness_Machine_to_Machine_model version: 2
IsolationforestTelstra_Enterprise_Postpaid_Handheld_model version: 2
Isolation

In [7]:
%%writefile $experiment_folder/python_env2.yml

name: python_env2

dependencies:
- python
- anaconda
- pip
- pip:
      - azureml.sdk
      - sklearn
      - statsmodels
      - pyarrow
      - numpy
      - pandas
      - seaborn
      - joblib
      - shap
      - graphviz 
      - tensorflow 
      - plotly
      - pmdarima
      - emmv
      
      
      
      
           

Overwriting batch_pipeline/python_env2.yml


In [8]:
from azureml.core import Dataset
SIO_data_salescube = Dataset.get_by_name(ws, name='SIO_salescube')
Activity_data_salescube = Dataset.get_by_name(ws, name='activity_salescube')
SIO_data_davinci = Dataset.get_by_name(ws, name='SIO_davinci')
Activity_data_davinci = Dataset.get_by_name(ws, name='activity_davinci')



In [9]:
# from azureml.core import Environment
# from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# # Create an Environment for the experiment
# batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/inference_python_env2.yml")
# #batch_env.docker.base_image = DEFAULT_CPU_IMAGE
# print('Configuration ready.')

In [10]:
from azureml.core import Environment
experiment_env = Environment.from_conda_specification("python_env2", experiment_folder + "/python_env2.yml")
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws,name="python_env2",version="18")

#registered_env = Environment.get(ws, 'python_env2')


No Python version provided, defaulting to "3.8.12"


In [11]:
from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.runconfig import RunConfiguration
pipeline_run_config = RunConfiguration()
pipeline_run_config.environment = registered_env

output_dir = OutputFileDatasetConfig(name='inferences')

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="ams_score.py",
    #mini_batch_size="7",
    error_threshold=30000,
    output_action="append_row",
    environment=registered_env,
    compute_target= inference_cluster,
    node_count=2)

parallelrun_step = ParallelRunStep(
    name='batch-score-ams',
    parallel_run_config=parallel_run_config,
    inputs=[SIO_data_salescube.as_named_input('SIO_salecube'),Activity_data_salescube.as_named_input('activity_salescube'),SIO_data_davinci.as_named_input('SIO_davinci'),Activity_data_davinci.as_named_input('activity_davinci')],
    output=output_dir,
    arguments=[],
    allow_reuse=False
)

teams_step = PythonScriptStep(name = "Teams",
                                source_directory = experiment_folder,
                                script_name = "team_script.py",
                                #arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = inference_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse=False)



print('Steps defined')

Steps defined


In [12]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

#sequence = [parallelrun_step,teams_step]
sequence = [parallelrun_step]
pipeline = Pipeline(workspace=ws, steps=[sequence])
pipeline_run = Experiment(ws, 'ams_batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)
RunDetails(pipeline_run).show()


Created step batch-score-ams [78166a72][d0d8b631-8eaa-430c-847c-74f3ba271139], (This step will run and generate new outputs)
Submitted PipelineRun fece6fa3-145c-476c-9bb1-545fcb73ddda
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fece6fa3-145c-476c-9bb1-545fcb73ddda?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854
PipelineRunId: fece6fa3-145c-476c-9bb1-545fcb73ddda
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fece6fa3-145c-476c-9bb1-545fcb73ddda?wsid=/subscriptions/c4f0bb45-07a9-489a-8f6a-6fbc64f25b46/resourcegroups/tcp-azu0044-ae-rg-dcml-uc001-dt/workspaces/tcp0044aedcmluc001dtmlws&tid=49dfc6a3-5fb7-49f4-adea-c54e725bb854
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 6693621c-10b1-481c-bff1-928cc06d190e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6693621c-10b1-481c-bff1-

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
# # Publish the pipeline from the run
# published_pipeline = pipeline_run.publish_pipeline(
#     name="AMS_anomaly_inference_pipeline_demo", description="AMS_all_models_inference", version="Latest")

# published_pipeline

# print(published_pipeline)

In [14]:
# from azureml.pipeline.core import ScheduleRecurrence, Schedule

# # Submit the Pipeline every Day at 
# recurrence = ScheduleRecurrence(frequency="Day", interval=1, time_of_day="16:48" )
# day_schedule = Schedule.create(ws, name="AMS_anomaly_Inference-daily-run", 
#                                   description="Daily_run",
#                                   pipeline_id="f631e781-09ea-47ce-8e94-4d745cf04f49",
#                                   experiment_name='AMS_batch_daily_schedule', 
#                                   recurrence=recurrence)

In [15]:
sch= Schedule.list(ws)
sch

NameError: name 'Schedule' is not defined

In [ ]:
#  from azureml.pipeline.core import Schedule
#  import azureml.core
#  from azureml.core import Workspace
    
#  # Load the workspace from the saved config file
#  ws = Workspace.from_config()
#  sch = Schedule.list(ws)[0] # I want to disable the first pipeline
#  Schedule.disable(sch)
#  sch